In [1]:
import pandas as pd
import time
import numpy as np
import os
import random
from datetime import datetime
import math

In [2]:
# Define the path for all of the folders.
ED_FOLDER_PATH = "E:/AI-VR dataset/MIMIC-IV ED"
CLINICAL_FOLDER_PATH = "E:/AI-VR dataset/MIMIC-IV Clinical Database"
CXR_FOLDER_PATH = "E:/AI-VR dataset/MIMIC-CXR-JPG/physionet.org/files/mimic-cxr-jpg/2.0.0"
EYEGAZE_FOLDER_PATH = "E:/AI-VR dataset/eye-gaze-data-for-chest-x-rays-1.0.0"
REFLACX_FOLDER_PATH = "E:/AI-VR dataset/reflacx-reports-and-eye-tracking-data-for-localization-of-abnormalities-in-chest-x-rays-1.0.0/reflacx-reports-and-eye-tracking-data-for-localization-of-abnormalities-in-chest-x-rays-1.0.0"
XAMI_MIMIC_PATH = "E:/AI-VR dataset/XAMI-MIMIC"

In [38]:
## Since the CXR is the main data we want to have.
CXR_meta_df = pd.read_csv("E:/AI-VR dataset/MIMIC-CXR-JPG/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-metadata.csv.gz", compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False)

C:\Users\mike8\anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [39]:
CXR_meta_df.head(5)

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN


In [13]:
print(f"We have {len(CXR_meta_df)} CXR-images")

We have 377110 CXR-images


In [14]:
all_subject_ids_in_cxr_meta  = set(list(CXR_meta_df['subject_id']))
print(f"Total {len(all_subject_ids_in_cxr_meta)} patients have CXR images")

Total 65379 patients have CXR images


In [23]:
patients_df = pd.read_csv(f"{CLINICAL_FOLDER_PATH}/core/patients.csv")

In [16]:
patients_df.head(5)

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000048,F,23,2126,2008 - 2010,NaN
1,10002723,F,0,2128,2017 - 2019,NaN
2,10003939,M,0,2184,2008 - 2010,NaN
3,10004222,M,0,2161,2014 - 2016,NaN
4,10005325,F,0,2154,2011 - 2013,NaN


In [17]:
print(f"Total {len(patients_df)} patients in MIMIC dataset")
print(f"{len(all_subject_ids_in_cxr_meta)/len(patients_df)*(1e+2):.2f}% of the patients have CXR image")

Total 382278 patients in MIMIC dataset
17.10% of the patients have CXR image


In [18]:
# Let's check if all the patients having CXR are in the core.patients table, so we can retrieve their age and gender.
all_patients_id = list(patients_df['subject_id'])
patients_not_in_patients_table = []
all_subject_ids_in_cxr_and_patients_table = []

for id in all_subject_ids_in_cxr_meta:
    if not (id in all_patients_id):
        patients_not_in_patients_table.append(id)
    else:
        all_subject_ids_in_cxr_and_patients_table.append(id)

KeyboardInterrupt: 

In [ ]:
print(f"We have {len(patients_not_in_patients_table)} not in the patients table")
print(f"Other {len(all_subject_ids_in_cxr_and_patients_table)} recorded in the patients table")
print(f"Missing portion is minor ({len(patients_not_in_patients_table)/ len(all_subject_ids_in_cxr_meta)*(1e+2):.2f}%)")

We have 227 not in the patients table
Other 65152 recorded in the patients table
Missing portion is minor (0.35%)


In [24]:
## We load icu and ed stays to check.
ICU_icustays_df = pd.read_csv(f"{CLINICAL_FOLDER_PATH}/icu/icustays.csv")
ED_edstays_df = pd.read_csv(f"{ED_FOLDER_PATH}/edstays.csv")


In [ ]:
all_subject_ids_in_edstays = set(list(ED_edstays_df['subject_id']))
all_subject_ids_in_icu_stays = set(list(ICU_icustays_df['subject_id']))
subject_id_count_in_both = sum(
    [id in all_subject_ids_in_icu_stays for id in all_subject_ids_in_edstays])

print(f"We have {len(all_subject_ids_in_edstays)} patients in ED and {len(all_subject_ids_in_icu_stays)} in ICU")
print(f"{subject_id_count_in_both} of them are in both ICU and ED, which is {subject_id_count_in_both/ len(all_subject_ids_in_edstays)*(1e+2):.2f}% of ED patiens and {subject_id_count_in_both/ len(all_subject_ids_in_icu_stays)*(1e+2):.2f}% of ICU patiens")


We have 216877 patients in ED and 53150 in ICU
33228 of them are in both ICU and ED, which is 15.32% of ED patiens and 62.52% of ICU patiens


In [ ]:
all_cxr_and_patients_subject_ids_in_both_ed_and_icu = []
all_cxr_and_patients_subject_ids_in_only_ed = []
all_cxr_and_patients_subject_ids_in_only_icu = []
all_cxr_and_patients_subject_ids_in_not_in_icu_or_ed = []

for id in all_subject_ids_in_cxr_and_patients_table:
    if (id in all_subject_ids_in_edstays) and (id in all_subject_ids_in_icu_stays):
        all_cxr_and_patients_subject_ids_in_both_ed_and_icu.append(id)
    elif (id in all_subject_ids_in_edstays):
        all_cxr_and_patients_subject_ids_in_only_ed.append(id)
    elif (id in all_subject_ids_in_icu_stays):
        all_cxr_and_patients_subject_ids_in_only_icu.append(id)
    else:
        all_cxr_and_patients_subject_ids_in_not_in_icu_or_ed.append(id)

In [ ]:
print(f"The patients in [CXR-meta, patients] table: {len(all_subject_ids_in_cxr_and_patients_table)}")
print(f"The patients in [CXR-meta, patients, edstays] table: {len(all_cxr_and_patients_subject_ids_in_only_ed)}")
print(f"The patients in [CXR-meta, patients, icustays] table: {len(all_cxr_and_patients_subject_ids_in_only_icu)}")
print(f"The patients in [CXR-meta, patients, edstays, icustays] table: {len(all_cxr_and_patients_subject_ids_in_both_ed_and_icu)}")

### We may need to remove the 
print(f"The patients exist in [CXR-meta, patients] but not in [edstays] or [icustays]: {len(all_cxr_and_patients_subject_ids_in_not_in_icu_or_ed)}")

The patients in [CXR-meta, patients] table: 65152
The patients in [CXR-meta, patients, edstays] table: 44903
The patients in [CXR-meta, patients, icustays] table: 2
The patients in [CXR-meta, patients, edstays, icustays] table: 20243
The patients exist in [CXR-meta, patients] but not in [edstays] or [icustays]: 4


In [ ]:
## Check what kind of patients are beling used in the EyeGaze and REFLACX.

eye_gaze_master_df = pd.read_csv(f"{EYEGAZE_FOLDER_PATH}/master_sheet.csv")
reflacx_meta_df =  pd.concat([pd.read_csv(f"{REFLACX_FOLDER_PATH}/main_data/metadata_phase_{i}.csv") for i in range(1,4)])

In [ ]:
eye_gaze_master_df.head(5)

,dicom_id,path,study_id,patient_id,stay_id,gender,anchor_age,image_top_pad,image_bottom_pad,image_left_pad,...,fracture__chx,lung_lesion__chx,lung_opacity__chx,no_finding__chx,pleural_effusion__chx,pleural_other__chx,pneumonia__chx,pneumothorax__chx,support_devices__chx,cxr_exam_indication
0,24c7496c-d7635dfe-b8e0b87f-d818affc-78ff7cf4,files/p15/p15628804/s58573295/24c7496c-d7635df...,58573295,15628804,33811834,F,20 - 30,86,86,448,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,___F with CHF and shortness of breath// ?Pulm...
1,78711a04-264d5305-d5feec9b-ebef1cec-fdc6db9c,files/p19/p19462352/s51900589/78711a04-264d530...,51900589,19462352,32954494,F,20 - 30,0,0,534,...,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,"___F with hypertension, tachycardia"
2,a770d8d6-7b6a62ff-815ab876-c81709a8-9a654a54,files/p11/p11255143/s50941783/a770d8d6-7b6a62f...,50941783,11255143,34005408,F,20 - 30,0,0,534,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,History of myocardial infarction. Shortness o...
3,8e457921-bc1af8aa-a65073c1-aaac8247-c5ceb780,files/p10/p10526322/s55981398/8e457921-bc1af8a...,55981398,10526322,36680301,F,20 - 30,0,0,534,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,___F with SOB
4,62fe5d5a-1806ee3c-f4e742fa-f2b036ea-d390057a,files/p12/p12055181/s59722264/62fe5d5a-1806ee3...,59722264,12055181,30138691,M,20 - 30,0,0,534,...,NaN,NaN,1.0,NaN,1.0,NaN,-1.0,NaN,NaN,"___M with DOE, SOB in supine position // eval..."


In [ ]:
reflacx_meta_df.head(5)

,id,split,eye_tracking_data_discarded,image,dicom_id,subject_id,image_size_x,image_size_y,Airway wall thickening,Atelectasis,...,Support devices,Wide mediastinum,Abnormal mediastinal contour,Acute fracture,Enlarged hilum,Hiatal hernia,High lung volume / emphysema,Interstitial lung disease,Lung nodule or mass,Pleural abnormality
0,P102R108387,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,0.0,0.0,...,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P102R379837,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,0.0,0.0,...,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P102R558314,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,4.0,0.0,...,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P102R765317,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,0.0,0.0,...,True,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P102R915878,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,0.0,0.0,...,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print(f"{len(eye_gaze_master_df.dicom_id)} instances and {len(set(eye_gaze_master_df.dicom_id))} unique CXR-images in REFLACX dataset")
print(f"{len(reflacx_meta_df.dicom_id)} instances and {len(set(reflacx_meta_df.dicom_id))} unique CXR-images in REFLACX dataset")

1083 instances and 1083 unique CXR-images in REFLACX dataset
3052 instances and 2616 unique CXR-images in REFLACX dataset


In [ ]:
all_subject_ids_in_eye_gaze = set(list(eye_gaze_master_df['patient_id']))
all_subject_ids_in_reflacx = set(list(reflacx_meta_df['subject_id']))

In [ ]:
print(f"EyeGaze dataset has {len(all_subject_ids_in_eye_gaze)} patients")
print(f"REFLACX dataset has {len(all_subject_ids_in_reflacx)} patients")


EyeGaze dataset has 1038 patients
REFLACX dataset has 2199 patients


In [ ]:
print(f"{sum([id in all_subject_ids_in_reflacx  for id in all_subject_ids_in_eye_gaze])} patients are in the both datasets.")

45 patients are in the both datasets.


In [ ]:
eye_gaze_subject_id_not_in_patient= [id for id in all_subject_ids_in_eye_gaze if not id in all_patients_id]
reflacx_subject_id_not_in_patient = [id for id in all_subject_ids_in_reflacx if not id in all_patients_id]
print(f"{len(eye_gaze_subject_id_not_in_patient)} patients in EyeGaze dataset are not in patients table")
print(f"{len(reflacx_subject_id_not_in_patient)} patients in REFLACX dataset are not in patients table")

2 patients in EyeGaze dataset are not in patients table
5 patients in REFLACX dataset are not in patients table


In [ ]:
eye_gaze_subject_id_not_in_ed= [id for id in all_subject_ids_in_eye_gaze if not id in all_subject_ids_in_edstays]
reflacx_subject_id_not_in_ed = [id for id in all_subject_ids_in_reflacx if not id in all_subject_ids_in_edstays]
print(f"{len(eye_gaze_subject_id_not_in_ed)} patients in EyeGaze dataset are not in ED table")
print(f"{len(reflacx_subject_id_not_in_ed)} patients in REFLACX dataset are not in ED table")


2 patients in EyeGaze dataset are not in ED table
6 patients in REFLACX dataset are not in ED table


In [ ]:
eye_gaze_subject_id_not_in_ed= [id for id in all_subject_ids_in_eye_gaze if not id in all_subject_ids_in_icu_stays]
reflacx_subject_id_not_in_ed = [id for id in all_subject_ids_in_reflacx if not id in all_subject_ids_in_icu_stays]
print(f"{len(eye_gaze_subject_id_not_in_ed)} patients in EyeGaze dataset are not in ICU table")
print(f"{len(reflacx_subject_id_not_in_ed)} patients in REFLACX dataset are not in ICU table")

745 patients in EyeGaze dataset are not in ED table
820 patients in REFLACX dataset are not in ED table


In [ ]:
all_ed_icu_subject_ids = set(list(all_subject_ids_in_edstays) + list(all_subject_ids_in_icu_stays))

eye_gaze_subject_id_not_in_ed= [id for id in all_subject_ids_in_eye_gaze if not id in all_ed_icu_subject_ids]
reflacx_subject_id_not_in_ed = [id for id in all_subject_ids_in_reflacx if not id in all_ed_icu_subject_ids]
print(f"{len(eye_gaze_subject_id_not_in_ed)} patients in EyeGaze dataset are not in both ED and ICU tables")
print(f"{len(reflacx_subject_id_not_in_ed)} patients in REFLACX dataset are not in both ED and ICU tables")


### ! Question: if we have 2 paitents in the EyeGaze dataset are not in the edstays table, how can the author identify the stay_id for them? 

2 patients in EyeGaze dataset are not in both ED and ICU tables
6 patients in REFLACX dataset are not in both ED and ICU tables


In [ ]:
## The stay_id is determined.
eye_gaze_master_df[[id in eye_gaze_subject_id_not_in_ed for id in eye_gaze_master_df['patient_id']]]

,dicom_id,path,study_id,patient_id,stay_id,gender,anchor_age,image_top_pad,image_bottom_pad,image_left_pad,...,fracture__chx,lung_lesion__chx,lung_opacity__chx,no_finding__chx,pleural_effusion__chx,pleural_other__chx,pneumonia__chx,pneumothorax__chx,support_devices__chx,cxr_exam_indication
280,2b42b3e0-9d6dd85a-80eb0158-0bbb96b6-8d5b4060,files/p11/p11055697/s58394099/2b42b3e0-9d6dd85...,58394099,11055697,38983684,F,70 - 80,0,0,534,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,"___F with DOE, orthopnea, substernal CP // ev..."
402,07886c95-b1f91525-6cc45c85-f0f9d171-2c319917,files/p13/p13784711/s51872217/07886c95-b1f9152...,51872217,13784711,30781437,M,20 - 30,86,86,448,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,High fever and productive cough.


In [ ]:
reflacx_meta_df[[id in reflacx_subject_id_not_in_ed for id in reflacx_meta_df['subject_id']]]

,id,split,eye_tracking_data_discarded,image,dicom_id,subject_id,image_size_x,image_size_y,Airway wall thickening,Atelectasis,...,Support devices,Wide mediastinum,Abnormal mediastinal contour,Acute fracture,Enlarged hilum,Hiatal hernia,High lung volume / emphysema,Interstitial lung disease,Lung nodule or mass,Pleural abnormality
586,P300R916588,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p12/...,2403a450-ac53ba33-79736f35-a5367b22-75c2b2a1,12518231,2544,3056,NaN,0.0,...,False,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
911,P300R627128,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p13/...,cab7bd95-1e76f5d7-eba32616-1e1e9b95-1f131be0,13906874,2544,3056,NaN,0.0,...,False,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1138,P300R578212,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p14/...,804fd3f5-bf769e56-79b96d6c-61d59859-75611bea,14886832,2544,3056,NaN,0.0,...,False,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1457,P300R131617,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p15/...,f2b93f5f-e2091beb-adb92de4-ac556360-4d8f6bef,15959458,3056,2544,NaN,0.0,...,True,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1885,P300R709087,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p17/...,d05c7f9a-16a22c62-0f7e0253-b724615a-45a98a66,17659584,2539,2424,NaN,0.0,...,False,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2456,P300R322446,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p19/...,609c6194-dbb9113d-84b48330-3c77c0ff-41d0df50,19787175,2544,3056,NaN,0.0,...,False,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ED_edstays_df[(ED_edstays_df['stay_id'] == "38983684")]

,subject_id,hadm_id,stay_id,intime,outtime


In [ ]:
ED_edstays_df[(ED_edstays_df['stay_id'] == "30781437")]

,subject_id,hadm_id,stay_id,intime,outtime


In [ ]:
ICU_icustays_df[(ICU_icustays_df['stay_id'] == "38983684")]


,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los


In [ ]:
ICU_icustays_df[(ICU_icustays_df['stay_id'] == "30781437")]


,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los


In [ ]:
# ! How does the author indentify these two?

In [ ]:
eye_gaze_master_df.columns

Index(['dicom_id', 'path', 'study_id', 'patient_id', 'stay_id', 'gender',
       'anchor_age', 'image_top_pad', 'image_bottom_pad', 'image_left_pad',
       'image_right_pad', 'dx1', 'dx1_icd', 'dx2', 'dx2_icd', 'dx3', 'dx3_icd',
       'dx4', 'dx4_icd', 'dx5', 'dx5_icd', 'dx6', 'dx6_icd', 'dx7', 'dx7_icd',
       'dx8', 'dx8_icd', 'dx9', 'dx9_icd', 'normal_reports', 'Normal', 'CHF',
       'pneumonia', 'consolidation', 'enlarged_cardiac_silhouette',
       'linear__patchy_atelectasis', 'lobar__segmental_collapse',
       'not_otherwise_specified_opacity___pleural__parenchymal_opacity__',
       'pleural_effusion_or_thickening', 'pulmonary_edema__hazy_opacity',
       'normal_anatomically', 'elevated_hemidiaphragm', 'hyperaeration',
       'vascular_redistribution', 'atelectasis__chx', 'cardiomegaly__chx',
       'consolidation__chx', 'edema__chx', 'enlarged_cardiomediastinum__chx',
       'fracture__chx', 'lung_lesion__chx', 'lung_opacity__chx',
       'no_finding__chx', 'pleural_effu

In [ ]:
reflacx_meta_df.head(5)

,id,split,eye_tracking_data_discarded,image,dicom_id,subject_id,image_size_x,image_size_y,Airway wall thickening,Atelectasis,...,Support devices,Wide mediastinum,Abnormal mediastinal contour,Acute fracture,Enlarged hilum,Hiatal hernia,High lung volume / emphysema,Interstitial lung disease,Lung nodule or mass,Pleural abnormality
0,P102R108387,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,0.0,0.0,...,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P102R379837,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,0.0,0.0,...,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P102R558314,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,4.0,0.0,...,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P102R765317,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,0.0,0.0,...,True,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P102R915878,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,0.0,0.0,...,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
f"{CXR_FOLDER_PATH}/"

In [ ]:
reflacx_meta_df =  pd.concat([pd.read_csv(f"{REFLACX_FOLDER_PATH}/main_data/metadata_phase_{i}.csv") for i in range(1,4)])


In [ ]:
# for each instance in the reflacx, we go to CXR meta to find the timestamp of this study.

reflacx_cannot_find_stay = []

ED_edstays_df['intime_float'] = ED_edstays_df.intime.apply(lambda x: float(x.replace("-", "").replace(" ", "").replace(":", "")))
ED_edstays_df['outtime_float'] = ED_edstays_df.outtime.apply(lambda x: float(x.replace("-", "").replace(" ", "").replace(":", "")))
reflacx_meta_df['study_time'] = [None] * len(reflacx_meta_df)

for idx, dataset_instance in enumerate(reflacx_meta_df.iloc):
    dicom_id = dataset_instance['dicom_id']
    id_matches = CXR_meta_df[CXR_meta_df['dicom_id'] == dicom_id]

    if (len(id_matches) != 1):
        print(f"Matching count is {len(CXR_meta_instance)} for {dicom_id} CXR image in CXR-meta")
        reflacx_cannot_find_stay.append(dataset_instance)
        continue

    CXR_meta_instance = id_matches.iloc[0]
    study_time = float(str(CXR_meta_instance['StudyDate']) + str(CXR_meta_instance['StudyTime']))
    reflacx_meta_df.at[idx, 'study_time'] = study_time

    stay_time_match = ED_edstays_df[
        (ED_edstays_df.subject_id == dataset_instance.subject_id) &
        (ED_edstays_df.intime_float < study_time) &
        (ED_edstays_df.outtime_float > study_time)
    ]

    if (len(stay_time_match) == 1):
        stay_id = stay_time_match.iloc[0]['stay_id']
        reflacx_meta_df.at[idx, "stay_id"] = stay_id
        print(f"{dicom_id} pass!")
    else:
        print(f"Having problem with {dicom_id} CXR iamge, it has {len(stay_time_match)} matches in the edstays")
        reflacx_cannot_find_stay.append(dataset_instance)
        continue


34cedb74-d0996b40-6d218312-a9174bea-d48dc033 pass!
34cedb74-d0996b40-6d218312-a9174bea-d48dc033 pass!
34cedb74-d0996b40-6d218312-a9174bea-d48dc033 pass!
34cedb74-d0996b40-6d218312-a9174bea-d48dc033 pass!
34cedb74-d0996b40-6d218312-a9174bea-d48dc033 pass!
Having problem with ecaab183-6c08f6b8-21edcbae-0f9547f2-c28f62d0 CXR iamge, it has 0 matches in the edstays
Having problem with ecaab183-6c08f6b8-21edcbae-0f9547f2-c28f62d0 CXR iamge, it has 0 matches in the edstays
Having problem with ecaab183-6c08f6b8-21edcbae-0f9547f2-c28f62d0 CXR iamge, it has 0 matches in the edstays
Having problem with ecaab183-6c08f6b8-21edcbae-0f9547f2-c28f62d0 CXR iamge, it has 0 matches in the edstays
Having problem with ecaab183-6c08f6b8-21edcbae-0f9547f2-c28f62d0 CXR iamge, it has 0 matches in the edstays
Having problem with fb97dc99-52ef2345-cca09851-57c3d33d-c0fcf34c CXR iamge, it has 0 matches in the edstays
Having problem with fb97dc99-52ef2345-cca09851-57c3d33d-c0fcf34c CXR iamge, it has 0 matches in t

In [396]:
len(reflacx_cannot_find_stay) / len(reflacx_meta_df)

0.8126216291353906

In [3]:
transfers_df = pd.read_csv(f"{CLINICAL_FOLDER_PATH}/core/transfers.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'E:/AI-VR dataset/MIMIC-IV Clinical Database/core/transfers.csv'

In [29]:
transfers_df["eventtype"].unique()

array(['admit', 'discharge', 'ED', 'transfer'], dtype=object)

In [450]:
transfers_df[transfers_df["eventtype"] != 'discharge'].head(15)

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,intime_float,outtime_float
86,18342718,22405909.0,33253055,transfer,NaN,2157-03-22 09:32:54,2157-03-22 15:50:44,2.157032e+13,2.157032e+13
621,15875814,27950156.0,34601308,transfer,NaN,2127-05-30 10:01:15,2127-05-31 14:11:10,2.127053e+13,2.127053e+13
668,16838971,29663971.0,38088653,transfer,NaN,2170-03-18 16:56:33,2170-03-18 17:28:37,2.170032e+13,2.170032e+13
720,12422371,22216078.0,30171790,transfer,Medicine,2137-08-20 18:04:42,2137-08-22 16:51:35,2.137082e+13,2.137082e+13
792,10074117,27158113.0,36143470,transfer,Surgery/Trauma,2118-02-19 01:28:30,2118-02-19 19:10:01,2.118022e+13,2.118022e+13
858,13470788,23888583.0,38914819,transfer,NaN,2159-07-15 05:32:08,2159-07-15 05:32:54,2.159072e+13,2.159072e+13
870,17639750,21633533.0,35512800,transfer,NaN,2130-02-17 09:56:43,2130-02-18 13:08:01,2.130022e+13,2.130022e+13
955,12674049,28631387.0,37806325,transfer,NaN,2136-07-31 16:48:08,2136-07-31 17:12:50,2.136073e+13,2.136073e+13
1036,13330813,28409578.0,30765649,transfer,NaN,2172-04-22 14:38:11,2172-04-22 14:59:34,2.172042e+13,2.172042e+13
1086,17784861,21225919.0,31842829,transfer,NaN,2184-04-09 07:25:11,2184-04-10 14:18:56,2.184041e+13,2.184041e+13


In [294]:
eye_gaze_master_df.head(5)

,dicom_id,path,study_id,patient_id,stay_id,gender,anchor_age,image_top_pad,image_bottom_pad,image_left_pad,...,fracture__chx,lung_lesion__chx,lung_opacity__chx,no_finding__chx,pleural_effusion__chx,pleural_other__chx,pneumonia__chx,pneumothorax__chx,support_devices__chx,cxr_exam_indication
0,24c7496c-d7635dfe-b8e0b87f-d818affc-78ff7cf4,files/p15/p15628804/s58573295/24c7496c-d7635df...,58573295,15628804,33811834,F,20 - 30,86,86,448,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,___F with CHF and shortness of breath// ?Pulm...
1,78711a04-264d5305-d5feec9b-ebef1cec-fdc6db9c,files/p19/p19462352/s51900589/78711a04-264d530...,51900589,19462352,32954494,F,20 - 30,0,0,534,...,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,"___F with hypertension, tachycardia"
2,a770d8d6-7b6a62ff-815ab876-c81709a8-9a654a54,files/p11/p11255143/s50941783/a770d8d6-7b6a62f...,50941783,11255143,34005408,F,20 - 30,0,0,534,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,History of myocardial infarction. Shortness o...
3,8e457921-bc1af8aa-a65073c1-aaac8247-c5ceb780,files/p10/p10526322/s55981398/8e457921-bc1af8a...,55981398,10526322,36680301,F,20 - 30,0,0,534,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,___F with SOB
4,62fe5d5a-1806ee3c-f4e742fa-f2b036ea-d390057a,files/p12/p12055181/s59722264/62fe5d5a-1806ee3...,59722264,12055181,30138691,M,20 - 30,0,0,534,...,NaN,NaN,1.0,NaN,1.0,NaN,-1.0,NaN,NaN,"___M with DOE, SOB in supine position // eval..."


In [26]:
def time_string_to_float(x):
    return float('inf') if type(x) is float and math.isnan(x) else float(x.replace("-", "").replace("/", "").replace(" ", "").replace(":", ""))

In [33]:
# for each instance in the reflacx, we go to CXR meta to find the timestamp of this study.

eye_gaze_cannot_find_stay = []

ED_edstays_df['intime_float'] = ED_edstays_df.intime.apply(time_string_to_float)
ED_edstays_df['outtime_float'] = ED_edstays_df.outtime.apply(time_string_to_float)
ICU_icustays_df['intime_float']= ICU_icustays_df.intime.apply(time_string_to_float)
ICU_icustays_df['outtime_float']= ICU_icustays_df.outtime.apply(time_string_to_float)
transfers_df['intime_float']= transfers_df.intime.apply(time_string_to_float)
transfers_df['outtime_float']= transfers_df.outtime.apply(time_string_to_float)
eye_gaze_master_df['study_time'] = [None] * len(eye_gaze_master_df)

for idx, dataset_instance in enumerate(eye_gaze_master_df.iloc):
    dicom_id = dataset_instance['dicom_id']
    id_matches = CXR_meta_df[CXR_meta_df['dicom_id'] == dicom_id]

    if (len(id_matches) != 1):
        print(f"Matching count is {len(CXR_meta_instance)} for {dicom_id} CXR image in CXR-meta")
        eye_gaze_cannot_find_stay.append(dataset_instance)
        continue

    CXR_meta_instance = id_matches.iloc[0]
    study_time = float(str(CXR_meta_instance['StudyDate']) + str(CXR_meta_instance['StudyTime']))
    eye_gaze_master_df.at[idx, 'study_time'] = study_time

    ed_stay_time_match = ED_edstays_df[
        (ED_edstays_df.subject_id == dataset_instance.patient_id) &
        (ED_edstays_df.intime_float < study_time) &
        (ED_edstays_df.outtime_float > study_time)
    ]

    icu_stay_time_match = ICU_icustays_df[
        (ICU_icustays_df.subject_id == dataset_instance.patient_id) &
        (ICU_icustays_df.intime_float < study_time) &
        (ICU_icustays_df.outtime_float > study_time)
    ]

    transfer_stay_time_match = transfers_df[
        (transfers_df["eventtype"] != 'discharge') &
        (transfers_df.subject_id == dataset_instance.patient_id) &
        (transfers_df.intime_float < study_time) &
        (transfers_df.outtime_float > study_time)
    ]

    total_match_count = len(ed_stay_time_match) + len(icu_stay_time_match) + len(transfer_stay_time_match)

    if (total_match_count >= 1):
        for match in [ed_stay_time_match, icu_stay_time_match, transfer_stay_time_match]:
            if (len(match) >= 1):
                if ("stay_id" in match.columns):
                    stay_id = match.iloc[0]['stay_id']
                else:
                    stay_id = match.iloc[0]['transfer_id']
                eye_gaze_master_df.at[idx, "stay_id"] = stay_id
                print(f"{dicom_id} pass!")
    else:
        print(f"Having problem with {dicom_id} CXR iamge, it has {(total_match_count)} matches. {len(ed_stay_time_match)} in ED, {len(icu_stay_time_match)} in ICU and {len(transfer_stay_time_match)} in Transfer")
        eye_gaze_cannot_find_stay.append(dataset_instance)
        continue


24c7496c-d7635dfe-b8e0b87f-d818affc-78ff7cf4 pass!
24c7496c-d7635dfe-b8e0b87f-d818affc-78ff7cf4 pass!
78711a04-264d5305-d5feec9b-ebef1cec-fdc6db9c pass!
78711a04-264d5305-d5feec9b-ebef1cec-fdc6db9c pass!
a770d8d6-7b6a62ff-815ab876-c81709a8-9a654a54 pass!
a770d8d6-7b6a62ff-815ab876-c81709a8-9a654a54 pass!
8e457921-bc1af8aa-a65073c1-aaac8247-c5ceb780 pass!
8e457921-bc1af8aa-a65073c1-aaac8247-c5ceb780 pass!
62fe5d5a-1806ee3c-f4e742fa-f2b036ea-d390057a pass!
62fe5d5a-1806ee3c-f4e742fa-f2b036ea-d390057a pass!
64beda42-873c8a44-3d9d3073-33372d32-dee84707 pass!
64beda42-873c8a44-3d9d3073-33372d32-dee84707 pass!
Having problem with 748991dc-fb330cd1-fc203d00-b89b5ccb-7995b475 CXR iamge, it has 0 matches. 0 in ED, 0 in ICU and 0 in Transfer
e5d1f575-e201925f-3d0faf91-7c8d8e98-b2550d52 pass!
e5d1f575-e201925f-3d0faf91-7c8d8e98-b2550d52 pass!
f0266425-1de60bab-cabee003-da9aa11e-49a31ace pass!
f0266425-1de60bab-cabee003-da9aa11e-49a31ace pass!
6ff76a0c-49246bc5-a7c149d1-59dc38d9-c11b7ec7 pass!
6ff

In [35]:
len(eye_gaze_cannot_find_stay)

417

In [34]:
len(eye_gaze_cannot_find_stay) / len(eye_gaze_master_df)

0.3850415512465374

In [468]:
CXR_meta_df["StudyTime"].sort_values() ## 24 hrs.

123054         0.187
123053         0.187
123052         0.187
363929         0.796
363928         0.796
             ...    
10343     235958.625
84794     235958.781
125274    235959.203
257449    235959.437
257448    235959.437
Name: StudyTime, Length: 377110, dtype: float64

In [41]:
failed_dicom_id = "748991dc-fb330cd1-fc203d00-b89b5ccb-7995b475"

eye_gaze_master_df[eye_gaze_master_df['dicom_id'] ==  failed_dicom_id]

,dicom_id,path,study_id,patient_id,stay_id,gender,anchor_age,image_top_pad,image_bottom_pad,image_left_pad,...,lung_lesion__chx,lung_opacity__chx,no_finding__chx,pleural_effusion__chx,pleural_other__chx,pneumonia__chx,pneumothorax__chx,support_devices__chx,cxr_exam_indication,study_time
6,748991dc-fb330cd1-fc203d00-b89b5ccb-7995b475,files/p12/p12661994/s56611517/748991dc-fb330cd...,56611517,12661994,38297021,M,20 - 30,86,86,448,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,___-year-old man with cardiomegaly and weight ...,2179081275653.920898


In [42]:
got_stay_id_in_eye_gaze = eye_gaze_master_df[eye_gaze_master_df['dicom_id'] ==  failed_dicom_id].iloc[0]['stay_id']

In [43]:
got_stay_id_in_eye_gaze

38297021

In [44]:
print(f" In  ED: {got_stay_id_in_eye_gaze in set(ED_edstays_df['stay_id'])}")
print(f" In  ICU: {got_stay_id_in_eye_gaze in set(ICU_icustays_df['stay_id'])}")
print(f" In  Transfer: {got_stay_id_in_eye_gaze in set(transfers_df['transfer_id'])}")


 In  ED: False
 In  ICU: False
 In  Transfer: True


In [45]:
CXR_meta_df[CXR_meta_df['dicom_id']==failed_dicom_id]

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning
99964,748991dc-fb330cd1-fc203d00-b89b5ccb-7995b475,12661994,56611517,CHEST (PA AND LAT),PA,2544,3056,21790812,75653.921,CHEST (PA AND LAT),postero-anterior,Erect


In [46]:
transfers_df[transfers_df['transfer_id'] == got_stay_id_in_eye_gaze]

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime,intime_float,outtime_float
898444,18560579,25811730.0,38297021,ED,Emergency Department,2130-08-17 12:16:00,2130-08-17 16:52:00,2.130082e+13,2.130082e+13


In [47]:
transfers_df[transfers_df['transfer_id'] == got_stay_id_in_eye_gaze][["intime", "outtime"]]


,intime,outtime
898444,2130-08-17 12:16:00,2130-08-17 16:52:00


In [510]:
CXR_meta_df[CXR_meta_df['dicom_id']== failed_dicom_id][["StudyDate", "StudyTime"]]

,StudyDate,StudyTime
99964,21790812,75653.921


In [514]:
# Will transfer and ED share stay_id?

len(set(transfers_df["transfer_id"])) + len(set(ED_edstays_df["stay_id"]))


# They share stay_id
len(set(list(transfers_df["transfer_id"]) + list(ED_edstays_df["stay_id"])))

2638507

In [520]:
downloaded_edstays_df = pd.read_csv("edstays.csv.gz", compression='gzip')
downloaded_edstays_df[downloaded_edstays_df['stay_id'] == got_stay_id_in_eye_gaze]

,subject_id,hadm_id,stay_id,intime,outtime


In [391]:
ED_edstays_df[ED_edstays_df['stay_id'] == str(dataset_instance['stay_id'])]

,subject_id,hadm_id,stay_id,intime,outtime,study_time,intime_float,outtime_float


In [392]:
ICU_icustays_df[ICU_icustays_df['stay_id'] == str(dataset_instance['stay_id'])]

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los


In [306]:
CXR_meta_instance = id_matches.iloc[0]

In [307]:
study_time = float(str(CXR_meta_instance['StudyDate']) + str(CXR_meta_instance['StudyTime']))

In [309]:
study_time

2179081275653.921

In [342]:
CXR_meta_instance

dicom_id                                      748991dc-fb330cd1-fc203d00-b89b5ccb-7995b475
subject_id                                                                        12661994
study_id                                                                          56611517
PerformedProcedureStepDescription                                       CHEST (PA AND LAT)
ViewPosition                                                                            PA
Rows                                                                                  2544
Columns                                                                               3056
StudyDate                                                                         21790812
StudyTime                                                                        75653.921
ProcedureCodeSequence_CodeMeaning                                       CHEST (PA AND LAT)
ViewCodeSequence_CodeMeaning                                              postero-anterior

In [ ]:
### Not finding this stay_id in the edstays


In [321]:
# stay_id 38297021 => where is it? 

38297021

In [341]:
## It's not in ICU or ED

print(38297021 in set(ED_edstays_df['stay_id']))
print(38297021 in set(ICU_icustays_df['stay_id']))

False
False


In [400]:
transfers_df.head()

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,14550633,24512724.0,35906369,admit,NaN,2176-10-10 01:58:40,2176-10-10 10:00:30
1,15216953,25759433.0,37381511,admit,NaN,2173-06-14 00:08:04,2173-06-14 13:35:19
2,15776555,28490372.0,34638843,admit,NaN,2187-02-01 00:09:41,2187-02-01 14:25:07
3,12248661,23426497.0,39272104,discharge,NaN,2141-12-05 14:11:09,NaN
4,14261068,20086032.0,36182135,discharge,NaN,2144-09-13 14:14:50,NaN


In [401]:
int(38297021) in set(transfers_df.transfer_id)

True

In [330]:
ED_edstays_df

,subject_id,hadm_id,stay_id,intime,outtime,study_time,intime_float,outtime_float
0,10000115,NaN,38081480,2154-12-10 02:04:00,2154-12-10 02:16:00,None,2.154121e+13,2.154121e+13
1,10000115,NaN,30295111,2154-12-17 16:37:00,2154-12-17 17:38:00,None,2.154122e+13,2.154122e+13
2,10000473,NaN,33267868,2138-03-15 20:07:00,2138-03-15 20:26:00,None,2.138032e+13,2.138032e+13
3,10000764,27897940.0,35420907,2132-10-14 19:31:00,2132-10-14 23:32:59,None,2.132101e+13,2.132101e+13
4,10001038,NaN,34301067,2149-08-07 08:51:00,2149-08-07 08:55:00,None,2.149081e+13,2.149081e+13
...,...,...,...,...,...,...,...,...
448967,19998350,NaN,36522433,2136-03-21 07:33:00,2136-03-21 09:07:00,None,2.136032e+13,2.136032e+13
448968,19999068,21606769.0,35130402,2161-08-24 03:14:00,2161-08-24 05:26:00,None,2.161082e+13,2.161082e+13
448969,19999270,NaN,30502357,2184-02-11 21:11:00,2184-02-11 21:14:00,None,2.184021e+13,2.184021e+13
448970,19999270,NaN,36263682,2184-08-19 23:26:00,2184-08-20 02:47:00,None,2.184082e+13,2.184082e+13


In [320]:
ED_edstays_df[(ED_edstays_df.subject_id == dataset_instance.patient_id)]


,subject_id,hadm_id,stay_id,intime,outtime,study_time,intime_float,outtime_float
238354,12661994,26207228.0,37484600,2179-08-12 06:51:00,2179-08-12 12:00:00,None,2.179081e+13,2.179081e+13
238355,12661994,27310303.0,30470449,2180-10-09 11:57:00,2180-10-09 17:13:47,None,2.180101e+13,2.180101e+13
238356,12661994,28200112.0,38043999,2180-12-02 17:30:00,2180-12-03 01:55:40,None,2.180120e+13,2.180120e+13
238357,12661994,24971579.0,33513652,2182-02-15 17:58:00,2182-02-16 02:44:00,None,2.182022e+13,2.182022e+13


21790812065100.0

In [319]:
ED_edstays_df[(ED_edstays_df.subject_id == dataset_instance.patient_id)].iloc[0]["outtime_float"]

21790812120000.0

In [ ]:
    eye_gaze_master_df.at[idx, 'study_time'] = study_time

    stay_time_match = ED_edstays_df[
        (ED_edstays_df.subject_id == dataset_instance.patient_id) &
        (ED_edstays_df.intime_float < study_time) &
        (ED_edstays_df.outtime_float > study_time)
    ]

    if (len(stay_time_match) == 1):
        stay_id = stay_time_match.iloc[0]['stay_id']
        eye_gaze_master_df.at[idx, "stay_id"] = stay_id
        print(f"{dicom_id} pass!")
    else:
        print(f"Having problem with {dicom_id} CXR iamge, it has {len(stay_time_match)} matches in the edstays")
        eye_gaze_cannot_find_stay.append(dataset_instance)
        continue


In [177]:
### do the same process for the eye gaze.

In [178]:
study_time = float(str(CXR_meta_instance['StudyDate']) + str(CXR_meta_instance['StudyTime']))

In [179]:
study_time

21510714135820.547

In [204]:
subject_id_cond = ED_edstays_df['subject_id'] == dataset_instance['subject_id']
later_than_in_cond = ED_edstays_df["intime"].apply(lambda x: float(x.replace("-","").replace(" ","").replace(":","")) < study_time)
earlier_than_out_cond = ED_edstays_df["outtime"].apply(lambda x: float(x.replace("-","").replace(" ","").replace(":","")) > study_time)
for i in zip(subject_id_cond, later_than_in_cond, earlier_than_out_cond):
    print(all(i))
    raise StopIteration

False


StopIteration: 

In [200]:
ED_edstays_df['subject_id'] == dataset_instance['subject_id']

0         False
1         False
2         False
3         False
4         False
          ...  
448967    False
448968    False
448969    False
448970    False
448971    False
Name: subject_id, Length: 448972, dtype: bool

In [202]:
ED_edstays_df["intime"].apply(lambda x: float(x.replace("-","").replace(" ","").replace(":","")) < study_time)

0         False
1         False
2          True
3          True
4          True
          ...  
448967     True
448968    False
448969    False
448970    False
448971    False
Name: intime, Length: 448972, dtype: bool

In [184]:
ED_edstays_df.iloc[0]["intime"]

subject_id               10000115
hadm_id                       NaN
stay_id                  38081480
intime        2154-12-10 02:04:00
outtime       2154-12-10 02:16:00
Name: 0, dtype: object

In [267]:
stay_time_match = ED_edstays_df[
    (ED_edstays_df.subject_id == dataset_instance.subject_id) &
    (ED_edstays_df.intime.apply(lambda x: float(
        x.replace("-", "").replace(" ", "").replace(":", "")) < study_time)) &
    (ED_edstays_df.outtime.apply(lambda x: float(
        x.replace("-", "").replace(" ", "").replace(":", "")) > study_time))
]


if (len(stay_time_match) == 1):
    stay_id = stay_time_match.iloc[0]['stay_id']
    reflacx_meta_df.at[idx, "stay_id"] = stay_id
else:
    raise StopIteration(f"Having problem with {dicom_id} CXR iamge")


,subject_id,hadm_id,stay_id,intime,outtime
381182,18111516,23402975.0,32067002,2151-07-14 09:58:00,2151-07-14 18:41:00


In [230]:
ED_edstays_df.intime.apply(lambda x: float(x.replace("-","").replace(" ","").replace(":",""))< study_time)

0         False
1         False
2          True
3          True
4          True
          ...  
448967     True
448968    False
448969    False
448970    False
448971    False
Name: intime, Length: 448972, dtype: bool

In [239]:
ED_edstays_df.subject_id == dataset_instance.subject_id

0         False
1         False
2         False
3         False
4         False
          ...  
448967    False
448968    False
448969    False
448970    False
448971    False
Name: subject_id, Length: 448972, dtype: bool

In [263]:
patient_ed_test_df = ED_edstays_df[(ED_edstays_df.subject_id == dataset_instance.subject_id) | (ED_edstays_df.intime.apply(lambda x: True))]

In [264]:
patient_ed_test_df

,subject_id,hadm_id,stay_id,intime,outtime
0,10000115,NaN,38081480,2154-12-10 02:04:00,2154-12-10 02:16:00
1,10000115,NaN,30295111,2154-12-17 16:37:00,2154-12-17 17:38:00
2,10000473,NaN,33267868,2138-03-15 20:07:00,2138-03-15 20:26:00
3,10000764,27897940.0,35420907,2132-10-14 19:31:00,2132-10-14 23:32:59
4,10001038,NaN,34301067,2149-08-07 08:51:00,2149-08-07 08:55:00
...,...,...,...,...,...
448967,19998350,NaN,36522433,2136-03-21 07:33:00,2136-03-21 09:07:00
448968,19999068,21606769.0,35130402,2161-08-24 03:14:00,2161-08-24 05:26:00
448969,19999270,NaN,30502357,2184-02-11 21:11:00,2184-02-11 21:14:00
448970,19999270,NaN,36263682,2184-08-19 23:26:00,2184-08-20 02:47:00


In [216]:
patien_ed_test_df[patien_ed_test_df.intime.apply(lambda x: float(x.replace("-","").replace(" ","").replace(":",""))< study_time)]

,subject_id,hadm_id,stay_id,intime,outtime
381170,18111516,26500600.0,34608765,2149-07-11 16:43:00,2149-07-12 00:55:00
381171,18111516,24950505.0,32771013,2149-09-12 16:53:00,2149-09-13 00:05:00
381172,18111516,26165825.0,37677481,2149-10-19 17:49:00,2149-10-19 21:01:09
381173,18111516,24949327.0,37077760,2150-06-09 08:10:00,2150-06-09 14:07:07
381174,18111516,27670516.0,30938305,2150-08-09 11:15:00,2150-08-09 19:36:00
381175,18111516,29548083.0,38258571,2150-09-13 10:20:00,2150-09-13 14:04:00
381176,18111516,21303371.0,33671905,2150-10-04 18:07:00,2150-10-04 23:07:00
381177,18111516,NaN,32737172,2150-12-17 15:29:00,2150-12-17 15:40:00
381178,18111516,28028511.0,34630683,2150-12-22 15:03:00,2150-12-22 21:40:35
381179,18111516,26564822.0,34800729,2151-02-14 00:46:00,2151-02-14 06:33:00


In [217]:
study_time

21510714135820.547

In [210]:
patient_ed_df

,subject_id,hadm_id,stay_id,intime,outtime
381170,18111516,26500600.0,34608765,2149-07-11 16:43:00,2149-07-12 00:55:00
381171,18111516,24950505.0,32771013,2149-09-12 16:53:00,2149-09-13 00:05:00
381172,18111516,26165825.0,37677481,2149-10-19 17:49:00,2149-10-19 21:01:09
381173,18111516,24949327.0,37077760,2150-06-09 08:10:00,2150-06-09 14:07:07
381174,18111516,27670516.0,30938305,2150-08-09 11:15:00,2150-08-09 19:36:00
381175,18111516,29548083.0,38258571,2150-09-13 10:20:00,2150-09-13 14:04:00
381176,18111516,21303371.0,33671905,2150-10-04 18:07:00,2150-10-04 23:07:00
381177,18111516,NaN,32737172,2150-12-17 15:29:00,2150-12-17 15:40:00
381178,18111516,28028511.0,34630683,2150-12-22 15:03:00,2150-12-22 21:40:35
381179,18111516,26564822.0,34800729,2151-02-14 00:46:00,2151-02-14 06:33:00


In [209]:
patient_ed_df["intime"].apply(lambda x: float(x.replace("-","").replace(" ","").replace(":",""))< study_time)

381170    True
381171    True
381172    True
381173    True
381174    True
381175    True
381176    True
381177    True
381178    True
381179    True
381180    True
381181    True
381182    True
Name: intime, dtype: bool

In [182]:
ED_edstays_df[]

,subject_id,hadm_id,stay_id,intime,outtime
381170,18111516,26500600.0,34608765,2149-07-11 16:43:00,2149-07-12 00:55:00
381171,18111516,24950505.0,32771013,2149-09-12 16:53:00,2149-09-13 00:05:00
381172,18111516,26165825.0,37677481,2149-10-19 17:49:00,2149-10-19 21:01:09
381173,18111516,24949327.0,37077760,2150-06-09 08:10:00,2150-06-09 14:07:07
381174,18111516,27670516.0,30938305,2150-08-09 11:15:00,2150-08-09 19:36:00
381175,18111516,29548083.0,38258571,2150-09-13 10:20:00,2150-09-13 14:04:00
381176,18111516,21303371.0,33671905,2150-10-04 18:07:00,2150-10-04 23:07:00
381177,18111516,NaN,32737172,2150-12-17 15:29:00,2150-12-17 15:40:00
381178,18111516,28028511.0,34630683,2150-12-22 15:03:00,2150-12-22 21:40:35
381179,18111516,26564822.0,34800729,2151-02-14 00:46:00,2151-02-14 06:33:00


In [181]:
reflacx_meta_df.head(5)

,id,split,eye_tracking_data_discarded,image,dicom_id,subject_id,image_size_x,image_size_y,Airway wall thickening,Atelectasis,...,Wide mediastinum,Abnormal mediastinal contour,Acute fracture,Enlarged hilum,Hiatal hernia,High lung volume / emphysema,Interstitial lung disease,Lung nodule or mass,Pleural abnormality,study_time
0,P102R108387,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.151071e+13
1,P102R379837,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P102R558314,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,4.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P102R765317,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,0.0,0.0,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P102R915878,train,False,physionet.org/files/mimic-cxr/2.0.0/files/p18/...,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,2544,3056,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
ED_edstays_df[ED_edstays_df['']]

,subject_id,hadm_id,stay_id,intime,outtime
0,10000115,NaN,38081480,2154-12-10 02:04:00,2154-12-10 02:16:00
1,10000115,NaN,30295111,2154-12-17 16:37:00,2154-12-17 17:38:00
2,10000473,NaN,33267868,2138-03-15 20:07:00,2138-03-15 20:26:00
3,10000764,27897940.0,35420907,2132-10-14 19:31:00,2132-10-14 23:32:59
4,10001038,NaN,34301067,2149-08-07 08:51:00,2149-08-07 08:55:00
...,...,...,...,...,...
448967,19998350,NaN,36522433,2136-03-21 07:33:00,2136-03-21 09:07:00
448968,19999068,21606769.0,35130402,2161-08-24 03:14:00,2161-08-24 05:26:00
448969,19999270,NaN,30502357,2184-02-11 21:11:00,2184-02-11 21:14:00
448970,19999270,NaN,36263682,2184-08-19 23:26:00,2184-08-20 02:47:00


In [ ]:
# Identify the stay_id for REFLACX


In [ ]:
# Create a related_stay_id list
# Create a related_subject_id list 

In [ ]:
#####################################################

In [8]:
ED_edstays_df.head(5)

,subject_id,hadm_id,stay_id,intime,outtime
0,10000115,NaN,38081480,2154-12-10 02:04:00,2154-12-10 02:16:00
1,10000115,NaN,30295111,2154-12-17 16:37:00,2154-12-17 17:38:00
2,10000473,NaN,33267868,2138-03-15 20:07:00,2138-03-15 20:26:00
3,10000764,27897940.0,35420907,2132-10-14 19:31:00,2132-10-14 23:32:59
4,10001038,NaN,34301067,2149-08-07 08:51:00,2149-08-07 08:55:00


In [21]:
# Check if every subject_id in the cxr_meta is in the edstays (or we have to check the icu stays)
all_subject_ids_in_edstays = set(list(ED_edstays_df['subject_id']))
all_subject_ids_in_cxr_meta  = set(list(cxr_meta_df['subject_id']))

not_in_edstays = []
id_in_edstays = []

for id in all_subject_ids_in_cxr_meta:
    if not (id in all_subject_ids_in_edstays):
        not_in_edstays.append(id)
        print(f"Found {id} not in the edstays table")
    else:
        id_in_edstays.append(id)
        

Found 14286871 not in the edstays table
Found 11403826 not in the edstays table
Found 14944478 not in the edstays table
Found 16124721 not in the edstays table
Found 14683579 not in the edstays table
Found 10883814 not in the edstays table
Found 19534625 not in the edstays table
Found 17175632 not in the edstays table
Found 13112524 not in the edstays table
Found 11409607 not in the edstays table
Found 18094583 not in the edstays table
Found 14162987 not in the edstays table
Found 12723036 not in the edstays table
Found 11674564 not in the edstays table
Found 15476275 not in the edstays table
Found 14036914 not in the edstays table
Found 14431042 not in the edstays table
Found 13251451 not in the edstays table
Found 13906874 not in the edstays table
Found 14037995 not in the edstays table
Found 17052771 not in the edstays table
Found 17708427 not in the edstays table
Found 12596787 not in the edstays table
Found 19938107 not in the edstays table
Found 18103164 not in the edstays table


In [34]:
id_in_edstays.sort()

In [39]:
len(id_in_edstays)

65146

In [24]:
len(not_in_edstays) # check if they're in the icu stays

233

In [25]:
ICU_icustays_df = pd.read_csv(f"{CLINICAL_FOLDER_PATH}/icu/icustays.csv")

In [26]:
ICU_icustays_df.head(5)

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,17867402,24528534,31793211,Trauma SICU (TSICU),Trauma SICU (TSICU),3/3/2154 4:11,3/4/2154 18:16,1.587454
1,14435996,28960964,31983544,Trauma SICU (TSICU),Trauma SICU (TSICU),6/19/2150 17:57,6/22/2150 18:33,3.025625
2,17609946,27385897,33183475,Trauma SICU (TSICU),Trauma SICU (TSICU),2/5/2138 18:54,2/15/2138 12:42,9.741725
3,18966770,23483021,34131444,Trauma SICU (TSICU),Trauma SICU (TSICU),10/25/2123 10:35,10/25/2123 18:59,0.350544
4,12776735,20817525,34547665,Neuro Stepdown,Neuro Stepdown,7/12/2200 0:33,7/13/2200 16:44,1.674769


In [27]:
all_subject_ids_in_icu_stays = set(list(ICU_icustays_df['subject_id']))

In [28]:
not_in_icu_too = []
not_in_edstays_but_icu = []

for id in not_in_edstays:
    if not (id in all_subject_ids_in_icu_stays):
        print(f"Found {id} not is ICU table as well")
        not_in_icu_too.append(id)
    else:
        not_in_edstays_but_icu.append(id)

Found 14286871 not is ICU table as well
Found 11403826 not is ICU table as well
Found 14944478 not is ICU table as well
Found 16124721 not is ICU table as well
Found 14683579 not is ICU table as well
Found 10883814 not is ICU table as well
Found 19534625 not is ICU table as well
Found 17175632 not is ICU table as well
Found 13112524 not is ICU table as well
Found 11409607 not is ICU table as well
Found 18094583 not is ICU table as well
Found 14162987 not is ICU table as well
Found 12723036 not is ICU table as well
Found 11674564 not is ICU table as well
Found 15476275 not is ICU table as well
Found 14036914 not is ICU table as well
Found 14431042 not is ICU table as well
Found 13251451 not is ICU table as well
Found 13906874 not is ICU table as well
Found 14037995 not is ICU table as well
Found 17052771 not is ICU table as well
Found 17708427 not is ICU table as well
Found 12596787 not is ICU table as well
Found 19938107 not is ICU table as well
Found 18103164 not is ICU table as well


In [38]:
len(not_in_icu_too)

231

In [31]:
not_in_icu_too.sort()

In [32]:
not_in_icu_too # start with 10xxxx

[10027672,
 10182178,
 10200894,
 10202793,
 10264902,
 10321217,
 10330900,
 10334183,
 10399798,
 10401281,
 10429629,
 10430258,
 10433079,
 10521158,
 10543848,
 10570315,
 10579365,
 10697585,
 10714268,
 10737302,
 10743215,
 10787743,
 10805247,
 10858283,
 10883814,
 10935675,
 10939057,
 10987456,
 11040153,
 11055697,
 11099310,
 11114794,
 11229078,
 11237430,
 11328203,
 11340250,
 11403826,
 11409607,
 11476176,
 11479501,
 11485348,
 11529815,
 11550925,
 11647826,
 11674564,
 11696247,
 11716769,
 11725509,
 11733756,
 11879340,
 11969490,
 11976834,
 12056844,
 12080215,
 12143281,
 12170289,
 12308109,
 12351520,
 12479021,
 12506591,
 12518231,
 12519260,
 12544972,
 12596787,
 12656203,
 12677162,
 12723036,
 12817225,
 12921089,
 12952847,
 12965241,
 13112524,
 13131199,
 13144491,
 13174181,
 13236615,
 13251451,
 13351653,
 13365335,
 13417910,
 13419758,
 13499619,
 13545082,
 13552561,
 13724843,
 13747454,
 13784711,
 13793679,
 13906874,
 13918030,
 13977850,

In [30]:
not_in_edstays_but_icu

[12338836, 11173476]

In [16]:
all_subject_ids_in_cxr_meta

{10000032,
 10000764,
 10000898,
 10000935,
 10000980,
 10001038,
 10001122,
 10001176,
 10001217,
 10001401,
 10001851,
 10001884}

In [14]:
id_in_edstays

[10000032,
 10001217,
 10000898,
 10001122,
 10000935,
 10001038,
 10001884,
 10000980,
 10001176,
 10001401,
 10001851,
 10000764]

In [12]:
not_in_edstays

[]

In [18]:
temp_df = pd.read_csv(f"{ED_FOLDER_PATH}/edstays.csv")

In [19]:
temp_df.head(5)

,subject_id,hadm_id,stay_id,intime,outtime
0,10000115,NaN,38081480,2154-12-10 02:04:00,2154-12-10 02:16:00
1,10000115,NaN,30295111,2154-12-17 16:37:00,2154-12-17 17:38:00
2,10000473,NaN,33267868,2138-03-15 20:07:00,2138-03-15 20:26:00
3,10000764,27897940.0,35420907,2132-10-14 19:31:00,2132-10-14 23:32:59
4,10001038,NaN,34301067,2149-08-07 08:51:00,2149-08-07 08:55:00


In [24]:
temp_df['intime'][0] 

str

In [29]:
'2154-12-10 02:04:00'

'2154-12-10 02:04:00'

In [37]:
"%y-%m-%d"

'%y-%m-%d'

In [43]:
a = datetime.strptime('2154-12-10 02:04:00' , "%Y-%m-%d %H:%M:%S")

In [45]:
int("2154-12-10 02:04:00".replace("-","").replace(" ","").replace(":","")) # using interger to perform comparison will be quicker.

21541210020400

In [42]:
b = datetime.strptime('2157-12-10 02:04:00' , "%Y-%m-%d %H:%M:%S")

In [ ]:
c = datetime.strftime('')

In [2]:
## importing MIMIC Clinical
import os


# Create a folder to contain all XAMI-MIMIC data
os.makedirs(XAMI_MIMIC_PATH, exist_ok=True) 

### Grab the first df

### We only keept the things that related to the patient
 


In [ ]:
# CXR study_id can be matched to an ED stay_id (cxr.StudyDateTime BETWEEN ed.intime AND ed.outtime) (already applied in sql)
# StudyDate - An anonymized date for the radiographic study. All images from the same study will have the same date and time. Dates are anonymized, but chronologically consistent for each patient. Intervals between two scans have not been modified during de-identification.
# StudyTime - The time of the study in hours, minutes, seconds, and fractional seconds. The time of the study was not modified during de-identification.

In [14]:
import pandas as pd

df = pd.read_csv("E:/AI-VR dataset/MIMIC-CXR-JPG/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-chexpert.csv.gz", compression='gzip', nrows=100, error_bad_lines=False)

C:\Users\mike8\anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
df

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,10000032,53911762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,10000032,56699142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,10000764,57375967,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10002428,57887188,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN
96,10002428,57961288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,1.0
97,10002428,58581921,NaN,0.0,1.0,0.0,NaN,NaN,0.0,1.0,NaN,1.0,NaN,0.0,NaN,NaN
98,10002428,58601917,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,1.0


C:\Users\mike8\anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [49]:
df

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,02e759a5-72ab444e-581216c8-af8fea64-8b1a744b,10001884,55333410,CHEST (PA AND LAT),LATERAL,3056,2544,21301128,153624.234,CHEST (PA AND LAT),lateral,Erect
96,5b135035-4708084f-3bb6d230-9db72889-2aa90c14,10001884,55333410,CHEST (PA AND LAT),PA,3056,2544,21301128,153624.234,CHEST (PA AND LAT),postero-anterior,Erect
97,4c3fdd2f-79be0bc9-f5a0ed41-3c9dc58e-75a6d19a,10001884,55893591,NaN,LL,2022,1736,21300906,132753.000,CHEST (PA AND LAT),left lateral,Erect
98,8d4eb7a8-0d3c4f7e-fed33834-ef1c6ffd-e01d0967,10001884,55893591,NaN,PA,2022,1736,21300906,132753.000,CHEST (PA AND LAT),postero-anterior,Erect


In [57]:
float(str(df["StudyDate"][0]) + str(df["StudyTime"][0])) ## tranform to number.

21800506213014

In [8]:
df = pd.read_csv("E:/AI-VR dataset/MIMIC-CXR-JPG/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-negbio.csv.gz", compression='gzip', nrows=100, error_bad_lines=False)


C:\Users\mike8\anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
df

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,10000032,53911762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,10000032,56699142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,10000764,57375967,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10002428,57887188,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN
96,10002428,57961288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,0.0,1.0
97,10002428,58581921,NaN,0.0,1.0,0.0,NaN,NaN,0.0,1.0,NaN,1.0,NaN,0.0,NaN,NaN
98,10002428,58601917,1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,1.0


In [10]:
df = pd.read_csv("E:/AI-VR dataset/MIMIC-CXR-JPG/physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-split.csv.gz", compression='gzip', nrows=100, error_bad_lines=False)


C:\Users\mike8\anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
df

,dicom_id,study_id,subject_id,split
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,50414267,10000032,train
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,50414267,10000032,train
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,53189527,10000032,train
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,53189527,10000032,train
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,53911762,10000032,train
...,...,...,...,...
95,02e759a5-72ab444e-581216c8-af8fea64-8b1a744b,55333410,10001884,train
96,5b135035-4708084f-3bb6d230-9db72889-2aa90c14,55333410,10001884,train
97,4c3fdd2f-79be0bc9-f5a0ed41-3c9dc58e-75a6d19a,55893591,10001884,train
98,8d4eb7a8-0d3c4f7e-fed33834-ef1c6ffd-e01d0967,55893591,10001884,train


In [ ]:
# Pick the table with subject_id, stay_id and dicom_id. Can't be linked to the CXR image.
# Create a folder for each patient.

In [1]:
# no link btw the stay_id and the study_id.
# file/{starting_str_of_patient}/{patient_id}/{study_id}/{dicom_id}
# 